# Import Section
---

In [1]:
import regex as re
import shutil
import os
from subprocess import Popen
import subprocess


import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from ipywidgets import AppLayout, Button, Layout, Box, FloatText, Textarea, Dropdown, Label, IntSlider
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

# Widgets Control Section
---

In [2]:
class train_config_and_cmds_widgets():
    
#------------------#
# init all the
#------------------#        
    def __init__(self):
        
        self.tflite_file_loc = ""
        
        form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
        )
    
        ### data prepare ###
        self.A_dp = widgets.Textarea(value=r'C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\train\data', 
                                     placeholder='Type something', disabled=False, layout={'height': '40pt'})
        self.B_dp = widgets.Textarea(value=r'C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\validation\data', 
                                     placeholder='Type something', disabled=False, layout={'height': '40pt'})
        self.C_dp = widgets.Textarea(value=r'C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\raw\instances_train2017.json', 
                                     placeholder='Type something', disabled=False, layout={'height': '40pt'})
        self.D_dp = widgets.Textarea(value=r'C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\raw\instances_val2017.json', 
                                     placeholder='Type something', disabled=False, layout={'height': '40pt'})
        self.E_dp = widgets.Text(value=r'dataset\datasets\coco_person', placeholder='Type something', disabled=False)
        self.F_dp = widgets.Text(value='person', placeholder='Type something', disabled=False)
        self.G_dp = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        
        form_data_prepare_items = [
            Box([Label(value = 'Train Image Dir'),        self.A_dp], layout=form_item_layout),
            Box([Label(value = 'Val Image Dir'),          self.B_dp], layout=form_item_layout),
            Box([Label(value = 'Train Annotations File'), self.C_dp], layout=form_item_layout),
            Box([Label(value = 'Val Annotations File'),   self.D_dp], layout=form_item_layout),
            Box([Label(value = 'Output Dir'),             self.E_dp], layout=form_item_layout),
            Box([Label(value = 'Object Name'),            self.F_dp], layout=form_item_layout),
            Box([Label(value = 'Start to Execute'),       self.G_dp], layout=form_item_layout),
        ]
        
        self.form_data_prepare_cmd = Box(form_data_prepare_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))
        
        ### train ###
        self.A_ta = widgets.Text(value='vww_v2_96_035', placeholder='Type something', disabled=False)
        self.E_ta = Dropdown(options=['mobilenetv2', 'mobilenetv1', 'mobilenetv3'])
        self.F_ta = Dropdown(options=['rgb', 'grayscale'])
        self.B_ta = widgets.IntSlider(value=256, min=4, max=512, step=4)
        self.C_ta = widgets.IntSlider(value=96, min=64, max=192, step=32)
        self.D_ta = widgets.FloatSlider(value=0.35, min=0.1, max=1.5, step=0.05)
        
        self.H_ta = widgets.BoundedIntText(value=5, min=1, max=500, step=1)
        self.I_ta = widgets.Text(value='0.0001', placeholder='Type something', disabled=False)

        self.J_ta = Dropdown(value='1: Train model & Convert to tflite', 
                             options=['0: Show the data pictures only', '1: Train model & Convert to tflite', 
                                      '2: Convert to tflite'])
        self.L_ta = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        
        self.K_ta = widgets.BoundedIntText(value=50, min=0, max=300, step=1, disabled=False)
        self.fine_tune_LA = Label(value = 'Freezing Layers of Fine-Tuning')        
        self.K_ta_box = Box([self.fine_tune_LA, self.K_ta], layout=form_item_layout)
        
        self.M_ta = widgets.BoundedIntText(value=50, min=1, max=1000, step=1)
        self.fine_tune_LB = Label(value = 'Epochs of Fine-Tuning')        
        self.M_ta_box = Box([self.fine_tune_LB, self.M_ta], layout=form_item_layout)
        
        self.N_ta = widgets.Text(value='0.00001', placeholder='Type something', disabled=False)
        self.fine_tune_LC = Label(value = 'Learning Rate of Fine-Tuning')        
        self.N_ta_box = Box([self.fine_tune_LC, self.N_ta], layout=form_item_layout)
        
        
        form_train_items = [
            Box([Label(value = 'Project Name'),                          self.A_ta], layout=form_item_layout),
            Box([Label(value = 'Model Type'),                            self.E_ta], layout=form_item_layout),
            Box([Label(value = 'Color Mode'),                            self.F_ta], layout=form_item_layout),
            Box([Label(value = 'Batch Size'),                            self.B_ta], layout=form_item_layout),
            Box([Label(value = 'Image Size'),                            self.C_ta], layout=form_item_layout),
            Box([Label(value = 'Alpha'),                                 self.D_ta], layout=form_item_layout),
           
            Box([Label(value = 'Epochs'),                                self.H_ta], layout=form_item_layout),
            Box([Label(value = 'Learning Rate'),                         self.I_ta], layout=form_item_layout),
            Box([Label(value = 'Switch Mode'),                           self.J_ta], layout=form_item_layout),
            self.K_ta_box,
            self.M_ta_box,
            self.N_ta_box,
            Box([Label(value = 'Start to Execute'),                      self.L_ta], layout=form_item_layout),
        ]
        
        self.form_output_train_cmd = Box(form_train_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',
            align_items='stretch',
            width='100%',
        ))
        
        ### test ###
        self.A_tt = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.C_tt = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
      
        form_test_items = [ 
            Box([Label(value = 'The tflite test setting is derivative from train setting.')], layout=form_item_layout),
            Box([Label(value = 'Choose the tflite file'), self.A_tt], layout=form_item_layout),
            Box([Label(value = 'Start to Test'),          self.C_tt], layout=form_item_layout),
        ]
        
        self.form_output_test_cmd = Box(form_test_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        ### convert model cpp ###
        self.A_cm = widgets.Text(value='..\workspace\\vww_v2_96_035\\tflite_model', placeholder='Type something', disabled=False)
        self.B_cm = widgets.Text(value='vww_mobilenetv2_0.35_96_96_int8quant.tflite', placeholder='Type something', disabled=False)
        self.C_cm = widgets.Text(value='..\workspace\\vww_v2_96_035\\tflite_model\\c', placeholder='Type something', disabled=False)
        self.E_cm = widgets.Button(description='Setting', layout=Layout(width='30%', height='30px'), button_style='success')
        self.D_cm = widgets.Button(description='Run', layout=Layout(width='30%', height='30px'), button_style='success')
        self.G_cm = widgets.Checkbox(value=False, disabled=False, indent=False)
      
        form_convert_items_paths =  [
            Box([Label(value = 'Vela Compiler'),          self.G_cm], layout=form_item_layout),
            Box([Label(value = 'Choose the tflite file'), self.E_cm], layout=form_item_layout),
            Box([Label(value = 'Model Src Dir'),          self.A_cm], layout=form_item_layout),
            Box([Label(value = 'Model Src File'),         self.B_cm], layout=form_item_layout),
            Box([Label(value = 'Gen Src Dir'),            self.C_cm], layout=form_item_layout)
        ]
        
        form_convert_items = [
            Box(form_convert_items_paths, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',    
            border='dotted 3px lightblue',
            align_items='stretch',
            width='70%')),
            Box([Label(value = 'Convert to cpp & Vela'), self.D_cm], layout=form_item_layout),
        ]
        
        self.form_output_convert_cmd = Box(form_convert_items, layout=Layout(
            display='flex',
            flex_flow='column',
            justify_content ='center',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='70%',
        ))

#------------------#
# help functions section
#------------------#            
    def move_allfiles(self, src_folder, dst_folder):
        copy_num = 0
        
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  #copy whole folder
                shutil.move(fullpath, dst_folder)
                print("Copy finish: {}".format(f))
    
    def show_headline(self, output):
        html0= widgets.HTML(value = f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)
        
#------------------#
# Show main widgets which combine all widgets.
#------------------#    
    def show_main(self):   
        
        intro_text = 'Please Choose the setting of data prepare & train'
        htmlWidget = widgets.HTML(value = f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(htmlWidget)
        
        #Create an accordion and put the 2 boxes
        accordion = widgets.Accordion(children=[self.form_data_prepare_cmd,
                                                self.form_output_train_cmd,
                                                ]).add_class("parentstyle")
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, 'COCO Dataset Prepare')
        accordion.set_title(1, 'Configure the Training')
        
        # Create a box combining with 2 elements
        box_data_train = Box([accordion], layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
        #Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[box_data_train, self.form_output_test_cmd, self.form_output_convert_cmd]).add_class("parentstyle")
        tab_contents = ['Train', 'Test', 'Deployment']
        tab.titles = tab_contents
       
        
        #Add a custom style tag to the notebook, you can use dev tool to inspect the class names
        #display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        #accordion.set_title(0, 'Configure the Training')
        
        output_widgets = widgets.Output(layout=Layout(border = '1px solid green'))
        
        def act_para(A_dp, B_dp, C_dp, D_dp, E_dp, F_dp,
                     A_ta, B_ta, C_ta, D_ta, E_ta, F_ta, 
                     H_ta, I_ta, J_ta, K_ta, M_ta, N_ta,
                     A_cm, B_cm, C_cm, G_cm):
            
            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()
                
            if F_ta.count('rgb'):
                self.K_ta.layout.visibility = 'visible'
                self.fine_tune_LA.layout.visibility = 'visible'
                self.M_ta.layout.visibility = 'visible'
                self.fine_tune_LB.layout.visibility = 'visible'
                self.N_ta.layout.visibility = 'visible'
                self.fine_tune_LC.layout.visibility = 'visible'
            else:
                self.K_ta.layout.visibility = 'hidden'
                self.fine_tune_LA.layout.visibility = 'hidden'
                self.M_ta.layout.visibility = 'hidden'
                self.fine_tune_LB.layout.visibility = 'hidden'
                self.N_ta.layout.visibility = 'hidden'
                self.fine_tune_LC.layout.visibility = 'hidden'
        
#------------------#
# widgets.Accordion's interactive input with action function `act_para()`
#------------------#
        out_inter = widgets.interactive_output(act_para, {'A_dp': self.A_dp, 'B_dp': self.B_dp, 'C_dp': self.C_dp,
                                                          'D_dp': self.D_dp, 'E_dp': self.E_dp, 'F_dp': self.F_dp,
                                                          'A_ta': self.A_ta, 'B_ta': self.B_ta, 'C_ta': self.C_ta,
                                                          'D_ta': self.D_ta, 'E_ta': self.E_ta, 'F_ta': self.F_ta,
                                                          'H_ta': self.H_ta, 'I_ta': self.I_ta,
                                                          'J_ta': self.J_ta, 'K_ta': self.K_ta, 'M_ta': self.M_ta,
                                                          'N_ta': self.N_ta,
                                                          'A_cm': self.A_cm, 'B_cm': self.B_cm, 'C_cm': self.C_cm,
                                                          'G_cm': self.G_cm
                                                          })
        display(tab, out_inter)
        display(output_widgets)
        
#------------------#
# buttoms interactive sections
#------------------#


        def on_button_clicked_dataprepare(b):
                with output_widgets:
                    clear_output()
                    print("Coco VWW dataprepare. . .")
                    self.run_coco_dataprepare()       
        self.G_dp.on_click(on_button_clicked_dataprepare)

        def on_button_clicked_train(b):
                with output_widgets:
                    clear_output()
                    print("Train. . .")
                    self.run_train()       
        self.L_ta.on_click(on_button_clicked_train)
        
        def on_button_clicked_choose_tflite(b):
                with output_widgets:
                    clear_output()
                    self.choose_tflite()       
        self.E_cm.on_click(on_button_clicked_choose_tflite)
        
        def on_button_clicked_cpp(b):
                with output_widgets:
                    clear_output()
                    print("Convert to cpp & Vela. . .")
                    self.convert_tflu()       
        self.D_cm.on_click(on_button_clicked_cpp)
        
        def on_button_clicked_choose_tflite_test(b):
                with output_widgets:
                    clear_output()
                    self.choose_tflite()       
        self.A_tt.on_click(on_button_clicked_choose_tflite_test)
        
        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.run_test_tflite()           
        self.C_tt.on_click(on_button_clicked_test)

#------------------#
# execute functions sections, handle all the interactive events
#------------------#        
    def run_coco_dataprepare(self):
        
        %run create_coco_vww_tf_record.py --train_image_dir $self.A_dp.value --val_image_dir $self.B_dp.value \
        --train_annotations_file $self.C_dp.value --val_annotations_file $self.D_dp.value \
        --object_name $self.F_dp.value --output_dir $self.E_dp.value 

    def choose_tflite(self):
        
        path_ftflite = os.path.join(os.getcwd(), "workspace")
        f_tflite = FileChooser(path_ftflite)
        # Restrict navigation to /Users
        f_tflite.sandbox_path = os.getcwd()
        f_tflite.filter_pattern = ['*.tflite']
        f_tflite.title = f"<b><font color='lightblue'><font size=4>Choose the Tflite for Converting.</b>"
        display(f_tflite)
        
        def act_test():
            self.tflite_file_loc = f_tflite.selected
            work_dir_name = os.getcwd().split("\\")[-1]
            m_src_dir = r".." + f_tflite.selected_path.split(work_dir_name)[-1]
            m_src_tflite  = f_tflite.selected.split("\\")[-1]
            print("The chosen dir: {}".format(m_src_dir))
            print("The chosen tflite: {}".format(m_src_tflite))
            self.A_cm.value = m_src_dir
            self.B_cm.value = m_src_tflite
            if self.G_cm.value:
                self.C_cm.value = os.path.join(m_src_dir, "vela")
            else:
                self.C_cm.value = os.path.join(m_src_dir, "c")
            
            
            print("Finish!")
        evt = interact_manual(act_test)
        evt.widget.children[0].description = 'Set this file'  #because there are 3 parameter of the evt
        evt.widget.children[0].button_style = 'primary'
        
    def run_train(self):
        sw_num = int(self.J_ta.value.split(":")[0])
        dataset_name = self.E_dp.value.split("\\")[-1]
        
        %run train_vww.py --proj_name $self.A_ta.value --dataset $dataset_name \
        --model-prefix $self.E_ta.value --COLOR_MODE $self.F_ta.value --batch-size $self.B_ta.value \
        --input-height $self.C_ta.value --input-width $self.C_ta.value --alpha $self.D_ta.value \
        --epochs $self.H_ta.value --learning-rate $self.I_ta.value --switch_mode $sw_num\
        --fine_tune_at $self.K_ta.value  --epochs_fine $self.M_ta.value --learning-rate_fine $self.N_ta.value
        
        # Save the cmd's record.
        path_CMDrecord = os.path.join(os.getcwd(), "workspace", self.A_ta.value, "train_cmd_record.txt")
        with open(path_CMDrecord, "w") as file1:
            open_dir = "cd {} \n".format(os.getcwd())
            cmd = "python train_vww.py --proj_name {} --dataset {} --model-prefix {} --COLOR_MODE {} --batch-size {} \
--input-height {} --input-width {} --alpha {} --epochs {} --learning-rate {} --switch_mode {} \
--fine_tune_at {} --epochs_fine {} --learning-rate_fine {}".format(self.A_ta.value, dataset_name, self.E_ta.value,
                    self.F_ta.value, self.B_ta.value, self.C_ta.value, self.C_ta.value, self.D_ta.value,
                    self.H_ta.value, self.I_ta.value, sw_num, self.K_ta.value, self.M_ta.value, self.N_ta.value)
            file1.writelines([open_dir, cmd])
        
        print("Finish Train!!")
        
    def run_test_tflite(self):
        dataset_name = self.E_dp.value.split("\\")[-1]
        print("The tflite file: {} The testing validation dataset: {}".format(self.tflite_file_loc, self.E_dp.value))
        
        %run train_vww.py --proj_name $self.A_ta.value --dataset $dataset_name \
        --model-prefix $self.E_ta.value --COLOR_MODE $self.F_ta.value --batch-size $self.B_ta.value \
        --input-height $self.C_ta.value --input-width $self.C_ta.value --alpha $self.D_ta.value \
        --epochs $self.H_ta.value --learning-rate $self.I_ta.value --switch_mode $3\
        --fine_tune_at $self.K_ta.value  --epochs_fine $self.M_ta.value --learning-rate_fine $self.N_ta.value\
        --TFLITE_F $self.tflite_file_loc
        
        print("Finish !!")
        
    def convert_tflu(self):
       
        print("Create")
        %run exebat.py --SRC_DIR $self.A_cm.value --SRC_FILE $self.B_cm.value --GEN_DIR $self.C_cm.value --VELA_EN $self.G_cm.value
        

# Run Section
---
- The detail description of all the parameters and each step meaning is here [meaning](#id-train_evl_monitor)
- In this notebook step, you have alreay finish the dataset prepared. If not, please go to `image_dataset\create_data.ipynb`.

In [3]:
act = train_config_and_cmds_widgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of data prepare & train</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…